In [62]:
from pyexistdb import db
import pandas as pd 
from lxml import etree
#import xml.etree.ElementTree as etree

In [63]:
exdb = db.ExistDB("http://127.0.0.1:8080/exist", username = "admin", password = "")

In [3]:
impf_table = exdb.getDocument("result_text2.xml")

ExistDBException: result_text2.xml not found

In [4]:
root = etree.fromstring(impf_table)

NameError: name 'impf_table' is not defined

In [5]:
columns = ["Impfung", "Jahr", "Anzahl"]
dframe = pd.DataFrame(columns = columns)

for child in root:
    impf = child[0].text
    jahr = child[1].text
    anz = child[2].text

    dframe = dframe.append(pd.Series([impf, jahr, anz], index = columns),
                                   ignore_index = True)

In [6]:
dframe

,Impfung,Jahr,Anzahl
0,Diphtherie-Tetanus-Keuchhusten-Hib-Kinderlaehm...,1995,0
1,Diphtherie-Tetanus-Keuchhusten-Hib-Kinderlaehmung,1995,0
2,Diphtherie-Tetanus-Keuchhusten-Kinderlaehmung,1995,0
3,Diphtherie-Tetanus-Keuchhusten-Hib,1995,200
4,Hib (Haemophilus influenzae b),1995,3724
...,...,...,...
589,Grippe,2016,2336
590,H1N1,2016,0
591,Pneumokokken,2016,441
592,Meningokokken,2016,2320


In [64]:
test = exdb.query("for $x in doc(\"result_text2.xml\") return $x")
test

In [70]:
def QueryToTree(query):
    result = exdb.query("<result>{"+query+"}</result>")
    for root in result.results:
        print(etree.tostring(root,pretty_print=True).decode())
    return

In [71]:
QueryToTree("for $x in doc(\"result_text3.xml\")/root/impf return if($x/anzahl>10) then <test>{data($x/impfung)}</test> else <test></test>")
#return if ($x/@category="children")
#then <child>{data($x/title)}</child>
#else <adult>{data($x/title)}</adult>

<result xmlns:exist="http://exist.sourceforge.net/NS/exist">
        <test/>
        <test/>
        <test/>
        <test>Diphtherie-Tetanus-Keuchhusten-Hib</test>
        <test>Hib (Haemophilus influenzae b)</test>
        <test/>
        <test>Diphtherie-Tetanus-Keuchhusten</test>
        <test>Diphtherie-Tetanus</test>
        <test/>
        <test/>
        <test>Kinderlaehmung (Poliomyelitis)</test>
        <test/>
        <test/>
        <test/>
        <test/>
        <test>Masern-Mumps-Roeteln</test>
        <test>Roeteln</test>
        <test/>
        <test/>
        <test/>
        <test/>
        <test>FSME</test>
        <test/>
        <test/>
        <test/>
        <test/>
        <test/>
        <test/>
        <test/>
        <test/>
        <test>Diphtherie-Tetanus-Keuchhusten-Hib</test>
        <test>Hib (Haemophilus influenzae b)</test>
        <test/>
        <test>Diphtherie-Tetanus-Keuchhusten</test>
        <test>Diphtherie-Tetanus</test>
        <test/>
      

In [81]:
root = test.results[0]
root.tag
root.attrib
root.text

'\n'

In [73]:
QueryToTree("for $x in doc(\"result_text3.xml\") return $x")

<result xmlns:exist="http://exist.sourceforge.net/NS/exist">
        <root>
<impf>
                <impfung>Diphtherie-Tetanus-Keuchhusten-Hib-Kinderlaehmung-Hepatitis B </impfung>
                <jahr>1995</jahr>
                <anzahl>0</anzahl>
            </impf>
<impf>
                <impfung>Diphtherie-Tetanus-Keuchhusten-Hib-Kinderlaehmung</impfung>
                <jahr>1995</jahr>
                <anzahl>0</anzahl>
            </impf>
<impf>
                <impfung>Diphtherie-Tetanus-Keuchhusten-Kinderlaehmung</impfung>
                <jahr>1995</jahr>
                <anzahl>0</anzahl>
            </impf>
<impf>
                <impfung>Diphtherie-Tetanus-Keuchhusten-Hib</impfung>
                <jahr>1995</jahr>
                <anzahl>200</anzahl>
            </impf>
<impf>
                <impfung>Hib (Haemophilus influenzae b)</impfung>
                <jahr>1995</jahr>
                <anzahl>3724</anzahl>
            </impf>
<impf>
                <impfung>Diphthe

In [112]:
QueryToTree("for $x in doc(\"result_text2.xml\")/root/impf return $x")

b'<impf>\n        <impfung>Diphtherie-Tetanus-Keuchhusten-Hib-Kinderlaehmung-Hepatitis B </impfung>\n        <jahr>1995</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie-Tetanus-Keuchhusten-Hib-Kinderlaehmung</impfung>\n        <jahr>1995</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie-Tetanus-Keuchhusten-Kinderlaehmung</impfung>\n        <jahr>1995</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie-Tetanus-Keuchhusten-Hib</impfung>\n        <jahr>1995</jahr>\n        <anzahl>200</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Hib (Haemophilus influenzae b)</impfung>\n        <jahr>1995</jahr>\n        <anzahl>3724</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie-Tetanus-Kinderlaehmung </impfung>\n        <jahr>1995</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie-Tetanus-Keuchhusten</impf

In [116]:
QueryToTree("for $x in doc(\"result_text2.xml\")/root/impf where $x/anzahl<10 order by $x/impfung return $x")

b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>1995</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>1996</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>1997</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>1998</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>1999</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>2000</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>2001</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n        <impfung>Diphtherie</impfung>\n        <jahr>2002</jahr>\n        <anzahl>0</anzahl>\n    </impf>\n    '
b'<impf>\n      